Никитина Александра 
## Машинное обучение в бизнесе
### Урок 2. Профилирование пользователей. Сегментация: unsupervised learning (clustering, LDA/ARTM), supervised (multi/binary classification)

In [154]:
import pandas as pd

Наши новости

In [155]:
!pip list
# Проверяю наличие модуля в импортеруемых ранее. Модуля нет. 

Package                       Version
----------------------------- --------------------
alabaster                     0.7.12
anaconda-client               1.11.0
anaconda-navigator            2.3.1
anaconda-project              0.11.1
anyio                         3.5.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arrow                         1.2.2
astroid                       2.11.7
astropy                       5.1
atomicwrites                  1.4.0
attrs                         21.4.0
Automat                       20.2.0
autopep8                      1.6.0
Babel                         2.9.1
backcall                      0.2.0
backports.functools-lru-cache 1.6.4
backports.tempfile            1.0
backports.weakref             1.0.post1
bcrypt                        3.2.0
beautifulsoup4                4.11.1
binaryornot                   0.4.4
bitarray                      2.5.1
bkcharts                      0.2
blac

In [156]:
# Установлю модуль wget.
!pip install wget

In [157]:
# Импортирую.
import wget 

In [158]:
#!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1zrdSHN2tq_Hj3YdbwlM3jk87Oct42XpR' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1zrdSHN2tq_Hj3YdbwlM3jk87Oct42XpR" -O articles.csv && rm -rf /tmp/cookies.txt 
#!wget 'https://drive.google.com/uc?export=download&id=1Q97K9eGrvpbS4ut5CphZa--gJDRqQp2a' -O users_articles.csv

In [159]:
#url = 'https://docs.google.com/uc?export=download&confirm=$'
#wget.download(url, 'C:/Users/LiLith/anaconda/MLforBisenes/Lection2/Lection2/')

После нескольких неудачных попыток загрузить новости, проверила ссылку, она оказалась не рабочей.
РАботаем с предоставленной информацией на диске.

In [160]:
news = pd.read_csv("articles.csv")
print(news.shape)
news.head(3)

(27000, 2)


,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


Загрузим пользователей и списки последних прочитанных новостей

In [161]:
users = pd.read_csv("users_articles.csv")
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


Итак, нам нужно получить векторные представления пользователей на основе прочитанным ими новостей и самих новостей

### 1. Получаем векторные представления новостей

In [162]:
!pip install razdel pymorphy2

In [163]:
# предобработка текстов
import re

# Модуль re предоставляет полную поддержку выражениям, подобным Perl в Python. 
# Модуль re поднимает исключение re.error, если возникает ошибка при компиляции 
# или использовании регулярного выражения.
# Почитать: "https://pythonru.com/osnovy/modul-re-dlja-reguljarnyh-vyrazhenij-v-python"

import numpy as np
from gensim.corpora.dictionary import Dictionary

# Gensim – это библиотека с открытым исходным кодом на Python, 
# написанная Радимом Рехуреком, которая используется для тематического моделирования без учителя 
# и обработки естественного языка (NLP). Она предназначена для извлечения семантических тем из документов.
# Почитать: "https://pythonru.com/biblioteki/gensim"

from razdel import tokenize  # сегментация русскоязычного текста на токены и предложения https://github.com/natasha/razdel
import pymorphy2  # Морфологический анализатор

In [164]:
import nltk # "https://nlpub.ru/NLTK" библиотека для  обработки естественного языка
from nltk.corpus import stopwords # модуль для удаления стоп слов
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LiLith\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [165]:
stopword_ru = stopwords.words('russian')
print(f'количество стоп слов выявленных "nltk.corpus.stopwords" = {len(stopword_ru)}',
      f"\n{'*'*100}\n перечислим их:",
      f'\n{stopword_ru}')


количество стоп слов выявленных "nltk.corpus.stopwords" = 151 
****************************************************************************************************
 перечислим их: 
['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно',

In [166]:
#!wget 'https://drive.google.com/uc?export=download&id=17wVn5YPpMjHToctGgff_KfSeWcIIlf7c' -O stopwords.txt

Проблемма та же, ссылка не доступна, работаем с диском. 
Скачаем с диска фаил с дополнительными стоп словам и добавим их в наш словарь. получается всего 776 слов

In [167]:
with open('stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
    
# Функция strip() – это предопределенная библиотечная функция Python.
# Она используется для возврата копии исходной строки путем удаления начальных 
# и конечных пробелов, символов, переданных в функцию strip().
# Источник: https://pythonpip.ru/examples/funktsiya-strip-v-python

# Метод файла file.readlines() читает файловый объект file построчно, 
# пока не достигнет конца файла EOF и возвращает список, содержащий строки 
# или байтовые объекты файла в зависимости от режима, в котором открыт файл функцией open(). 
# Конечный символ новой строки \n сохраняется в строке.
# Источник: https://docs-python.ru/tutorial/metody-fajlovogo-obekta-potoka-python/metod-file-readlines/
    
stopword_ru += additional_stopwords
len(stopword_ru)

776

In [168]:
# импортирую библиотеку pymorphy2 - морфологический анализатор, разработанный на языке программирования Python.
import pymorphy2


In [169]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    '''
    # Выполним проверку принадлежит ли наш текст типу строка
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower() # преобразуем строку в нижний регистр
    text = text.strip('\n').strip('\r').strip('\t') # удалим все символы табуляции, переноса
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))
    
    # Функция sub() модуля re возвращает строку, полученную путем замены крайнего левого неперекрывающегося вхождения шаблона 
    # регулярного выражения pattern в строке string на строку замены repl. 
    # Если шаблон регулярного выражения не найден, строка возвращается без изменений.

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    text = re.sub('n', ' ', text)
    
    return text

# ПРоводится морфологический анализ
cache = {}
morph = pymorphy2.MorphAnalyzer() 

def lemmatization(text):    
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист лемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w) > 1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords = [i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [170]:
from tqdm import tqdm
tqdm.pandas()

# Запускаем очистку текста. Будет долго...
news['title'] = news['title'].progress_apply(lambda x: clean_text(x))

100%|██████████████████████████████████████████████████████████████████████████| 27000/27000 [00:16<00:00, 1664.38it/s]


In [171]:
news['title'].iloc[:10]

0    заместитель председателя правительства рф серг...
1    матч  финала кубка россии по футболу был приос...
2    форвард авангарда томаш заборский прокомментир...
3    главный тренер кубани юрий красножан прокоммен...
4    решением попечительского совета владивостокско...
5    ио главного тренера вячеслав буцаев прокоммент...
6    запорожский металлург дома потерпел разгромное...
7    сборная сша одержала победу над австрией со сч...
8    бывший защитник сборной россии дарюс каспарайт...
9    полузащитник цска зоран тошич после победы над...
Name: title, dtype: object

In [172]:
# Запускаем лемматизацию текста. Будет очень долго...
news['title'] = news['title'].progress_apply(lambda x: lemmatization(x))

100%|███████████████████████████████████████████████████████████████████████████| 27000/27000 [02:29<00:00, 180.32it/s]


In [173]:
news['title'].iloc[:10]

0    [заместитель, председатель, правительство, рф,...
1    [матч, финал, кубок, россия, футбол, приостано...
2    [форвард, авангард, томаш, заборский, прокомме...
3    [главный, тренер, кубань, юрий, красножанин, п...
4    [решение, попечительский, совет, владивостокск...
5    [ио, главное, тренер, вячеслав, буцай, прокомм...
6    [запорожский, металлург, дом, потерпеть, разгр...
7    [сборная, сша, одержать, победа, австрия, счёт...
8    [бывший, защитник, сборная, россия, дарюс, кас...
9    [полузащитник, цска, зоран, тошич, победа, тер...
Name: title, dtype: object

А теперь в 3 строчки обучим нашу модель

In [174]:
# сформируем список наших текстов
texts = list(news['title'].values)

# Создадим корпус из списка с текстами
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

## Запускаем обучение

In [175]:
N_topic = 10  # можно было менять

In [176]:
from gensim.models import LdaModel

# Обучаем модель на корпусе
lda = LdaModel(common_corpus, num_topics=N_topic, id2word=common_dictionary, passes=8)  # можно было менять

In [177]:
from gensim.test.utils import datapath

# Сохраняем модель на диск
temp_file = datapath("model.lda")
lda.save(temp_file)

In [178]:
# Загружаем обученную модель с диска
lda = LdaModel.load(temp_file)

In [179]:
# Создаем новый корпус документов, которые раньше не видели
other_texts = list(news['title'].iloc[:3])
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc] 

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'свой', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'провести', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'работа', 'сказать', 'заборский', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'мочь', 'играть', 'ещё', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'это', 'хороший']


[(0, 0.04158683),
 (1, 0.50290656),
 (2, 0.022465598),
 (6, 0.32597542),
 (8, 0.09590103)]

Обучили модель. Теперь 2 вопроса:

1. как выглядят наши темы
2. как получить для документа вектор значений (вероятности принадлежности каждой теме)

In [180]:
x = lda.show_topics(num_topics=N_topic, num_words=7, formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

# Печатаем только слова
for topic, words in topics_words:
    print(f"topic_{topic}: " + " ".join(words))

topic_0: город район дом москва место операция клетка
topic_1: газ компания млн млрд турист год сколково
topic_2: температура день фестиваль мс °с погода градус
topic_3: это россия год который российский страна мочь
topic_4: учёный который год человек исследование свой суд
topic_5: ракета корабль дело который аппарат погибнуть взрыв
topic_6: который это год свой всё весь время
topic_7: озеро казахстан армения киргизия азербайджан молдавия узбекистан
topic_8: год это рубль рост тыс человек мочь
topic_9: самолёт сша американский армия сила полёт российский


Очень неплохо - большинство тем вполне можно описать о чем они

Давайте напишем функцию, которая будет нам возвращать векторное представление новости

In [181]:
def get_lda_vector(lda, text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]

    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(N_topic):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [182]:
get_lda_vector(lda, news['title'].iloc[0])

array([0.        , 0.14136125, 0.        , 0.75589079, 0.        ,
       0.0964245 , 0.        , 0.        , 0.        , 0.        ])

In [183]:
topic_matrix = pd.DataFrame([get_lda_vector(lda, text) for text in news['title'].values])
topic_matrix.columns = [f'topic_{i}' for i in range(N_topic)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+[f'topic_{i}' for i in range(N_topic)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9
0,6,0.000000,0.141361,0.000000,0.755896,0.000000,0.096420,0.000000,0.0,0.000000,0.0
1,4896,0.000000,0.398203,0.000000,0.000000,0.099727,0.484552,0.000000,0.0,0.000000,0.0
2,4897,0.041625,0.502903,0.022466,0.000000,0.000000,0.000000,0.325929,0.0,0.095913,0.0
3,4898,0.000000,0.317919,0.000000,0.044433,0.000000,0.000000,0.629280,0.0,0.000000,0.0
4,4899,0.000000,0.211819,0.000000,0.615961,0.000000,0.152688,0.000000,0.0,0.000000,0.0


Прекрасно, мы получили вектора наших новостей! И даже умеем интерпретировать получившиеся темы.

Можно двигаться далее

### Следующий шаг - векторные представления пользователей

In [184]:
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [185]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[[f'topic_{i}' for i in range(N_topic)]].values))

In [186]:
doc_dict[293672]

array([0.19441798, 0.        , 0.        , 0.32901549, 0.        ,
       0.        , 0.40503678, 0.        , 0.06116788, 0.        ])

In [187]:
def get_user_embedding(user_articles_list, doc_dict, func):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = func(user_vector, axis=0)
    return user_vector

In [188]:
user_articles_list = users['articles'].iloc[33]

get_user_embedding(user_articles_list, doc_dict, np.mean)

array([0.01975708, 0.        , 0.        , 0.48076149, 0.20465016,
       0.06178324, 0.05316352, 0.        , 0.08346497, 0.09073654])

Теперь получим эмбединги для всех пользователей и проверим их качество на конкретной downstream-задаче

In [189]:
FUNC = np.mean

In [190]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, doc_dict, FUNC))])
user_embeddings.columns = [f'topic_{i}' for i in range(N_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(N_topic)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9
0,u105138,0.108279,0.005238,0.004867,0.193689,0.147195,0.114292,0.319763,0.005185,0.031577,0.059490
1,u108690,0.039186,0.016197,0.000000,0.403942,0.137427,0.093731,0.141554,0.000000,0.112335,0.051488
2,u108339,0.183687,0.018089,0.017154,0.250716,0.219782,0.151492,0.030518,0.000000,0.098465,0.025936


Датасет готов - можно попробовать обучить модель. Загрузим нашу разметку

In [191]:
#!wget 'https://drive.google.com/uc?export=download&id=1nWKteQgEr9Rl8CwTRY7N2f7igevNH7oK' -O users_churn.csv

In [192]:
target = pd.read_csv("users_churn.csv")
target.head(3)

,uid,churn
0,u107120,0
1,u102277,0
2,u102444,0


In [193]:
X = pd.merge(user_embeddings, target, 'left')
X.head(5)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,churn
0,u105138,0.108279,0.005238,0.004867,0.193689,0.147195,0.114292,0.319763,0.005185,0.031577,0.059490,0
1,u108690,0.039186,0.016197,0.000000,0.403942,0.137427,0.093731,0.141554,0.000000,0.112335,0.051488,1
2,u108339,0.183687,0.018089,0.017154,0.250716,0.219782,0.151492,0.030518,0.000000,0.098465,0.025936,1
3,u101138,0.009808,0.249915,0.000000,0.098249,0.014947,0.128223,0.276034,0.069571,0.120109,0.024024,0
4,u108248,0.051823,0.021947,0.059942,0.297247,0.162161,0.102422,0.178219,0.000000,0.114125,0.003256,1


In [194]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

In [195]:
# разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(N_topic)]], 
                                                    X['churn'], random_state=0)

In [196]:
model = LogisticRegression()
# обучим 
model.fit(X_train, y_train)

LogisticRegression()

In [197]:
# наши прогнозы для тестовой выборки
preds = model.predict_proba(X_test)[:, 1]
preds[:10]

array([0.09373385, 0.06565015, 0.62416632, 0.17788597, 0.17070843,
       0.02627144, 0.10671615, 0.00805228, 0.42866723, 0.25113207])

In [198]:
metrics_df = pd.DataFrame(columns=['model', 'thresh', 'F-Score', 'Precision', 'Recall', 'ROC AUC'])
metrics_df

,model,thresh,F-Score,Precision,Recall,ROC AUC


In [199]:
from sklearn.metrics import (f1_score, roc_auc_score, precision_score,
                             classification_report, precision_recall_curve, confusion_matrix)

In [200]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')                                                                        

Best Threshold=0.2821708593838175, F-Score=0.698, Precision=0.676, Recall=0.722


In [201]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.9384173498459213

In [202]:
metrics_df = metrics_df.append({
    'model': FUNC.__name__,
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)

metrics_df

C:\Temp\ipykernel_12412\1288159249.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


,model,thresh,F-Score,Precision,Recall,ROC AUC
0,mean,0.282171,0.698225,0.675573,0.722449,0.938417


2. Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог)

In [203]:
FUNC = np.median

In [204]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, doc_dict, FUNC))])
user_embeddings.columns = [f'topic_{i}' for i in range(N_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(N_topic)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9
0,u105138,0.101223,0.0,0.0,0.163493,0.153604,0.012325,0.343746,0.0,0.000000,0.012913
1,u108690,0.029793,0.0,0.0,0.371790,0.170975,0.043268,0.156383,0.0,0.090589,0.023562
2,u108339,0.151889,0.0,0.0,0.274529,0.204849,0.163785,0.024274,0.0,0.106149,0.013388


In [205]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,churn
0,u105138,0.101223,0.0,0.0,0.163493,0.153604,0.012325,0.343746,0.0,0.000000,0.012913,0
1,u108690,0.029793,0.0,0.0,0.371790,0.170975,0.043268,0.156383,0.0,0.090589,0.023562,1
2,u108339,0.151889,0.0,0.0,0.274529,0.204849,0.163785,0.024274,0.0,0.106149,0.013388,1


In [206]:
# разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(N_topic)]], 
                                                    X['churn'], random_state=0)

In [207]:
model = LogisticRegression()
# обучим 
model.fit(X_train, y_train)

LogisticRegression()

In [208]:
# наши прогнозы для тестовой выборки
preds = model.predict_proba(X_test)[:, 1]
preds[:10]

array([0.14131881, 0.17655087, 0.56239544, 0.26018257, 0.11990595,
       0.03653198, 0.20716147, 0.00940038, 0.26077519, 0.14848538])

In [209]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')                                                                        

Best Threshold=0.2789462003229127, F-Score=0.685, Precision=0.662, Recall=0.710


In [210]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.9405849177277749

In [211]:
metrics_df = metrics_df.append({
    'model': FUNC.__name__,
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)

metrics_df

C:\Temp\ipykernel_12412\1288159249.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


,model,thresh,F-Score,Precision,Recall,ROC AUC
0,mean,0.282171,0.698225,0.675573,0.722449,0.938417
1,median,0.278946,0.685039,0.661597,0.710204,0.940585



3. Повторить п.2, но используя уже не медиану, а max


In [212]:
FUNC = np.max

In [213]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, doc_dict, FUNC))])
user_embeddings.columns = [f'topic_{i}' for i in range(N_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(N_topic)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9
0,u105138,0.194418,0.031428,0.029203,0.405191,0.267161,0.349921,0.617135,0.031111,0.128297,0.219804
1,u108690,0.104951,0.076872,0.000000,0.681627,0.236211,0.382255,0.305379,0.000000,0.268391,0.190411
2,u108339,0.434781,0.060785,0.102922,0.425706,0.433731,0.313401,0.060914,0.000000,0.226767,0.093998


In [214]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,churn
0,u105138,0.194418,0.031428,0.029203,0.405191,0.267161,0.349921,0.617135,0.031111,0.128297,0.219804,0
1,u108690,0.104951,0.076872,0.000000,0.681627,0.236211,0.382255,0.305379,0.000000,0.268391,0.190411,1
2,u108339,0.434781,0.060785,0.102922,0.425706,0.433731,0.313401,0.060914,0.000000,0.226767,0.093998,1


In [215]:
# разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(N_topic)]], 
                                                    X['churn'], random_state=0)

In [216]:
model = LogisticRegression()
# обучим 
model.fit(X_train, y_train)

LogisticRegression()

In [217]:
# наши прогнозы для тестовой выборки
preds = model.predict_proba(X_test)[:, 1]
preds[:10]

array([4.06696244e-02, 5.94834455e-05, 8.44746465e-01, 7.97673744e-02,
       2.51535197e-01, 8.01081642e-03, 1.42436103e-02, 5.48826418e-04,
       2.62981373e-01, 1.38587157e-01])

In [218]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')                                                                        

Best Threshold=0.311775128081935, F-Score=0.589, Precision=0.612, Recall=0.567


In [219]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.9060526774812488

In [220]:
metrics_df = metrics_df.append({
    'model': FUNC.__name__,
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)

metrics_df

C:\Temp\ipykernel_12412\1288159249.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


,model,thresh,F-Score,Precision,Recall,ROC AUC
0,mean,0.282171,0.698225,0.675573,0.722449,0.938417
1,median,0.278946,0.685039,0.661597,0.710204,0.940585
2,amax,0.311775,0.588983,0.612335,0.567347,0.906053


4. *Воспользовавшись полученными знаниями из п.1, повторить пункт 2, но уже взвешивая новости по tfidf (взяв список новостей пользователя)
	- подсказка 1: нужно получить веса-коэффициенты для каждого документа. Не все документы одинаково информативны и несут какой-то положительный сигнал
	- подсказка 2: нужен именно idf, как вес.

In [221]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [222]:
users['articles_str'] = users['articles'].apply(lambda x: x.replace('[','').replace(']', '').replace(',', ''))

users['articles_str'].iloc[0]

'293672 293328 293001 293622 293126 1852'

In [223]:
tfidf = TfidfVectorizer()
tfidf.fit(users['articles_str'])

TfidfVectorizer()

In [224]:
idf = pd.DataFrame({'article_id': tfidf.get_feature_names_out(),
                    'idf': tfidf.idf_})

idf

,article_id,idf
0,10,8.888710
1,100,7.907880
2,1000,8.041412
3,1001,8.888710
4,1002,8.888710
...,...,...
14776,995,8.377884
14777,996,8.195562
14778,997,8.601027
14779,998,9.294175


In [225]:
def get_user_embedding_idf(user_articles_list, doc_dict):
    user_articles_list = eval(user_articles_list)
    
    user_vector = np.zeros((len(user_articles_list), N_topic))
    for i, doc_id in enumerate(user_articles_list):
        try:
            weight = idf[idf['article_id'] == str(doc_id)]['idf'].values[0]
        except Exception as e:
            weight = 0
        user_vector[i] = doc_dict[doc_id] * weight

    user_vector = np.median(user_vector, axis=0)
    return user_vector

In [226]:
from tqdm import tqdm
tqdm.pandas()

user_embeddings = pd.DataFrame([i for i in users['articles'].progress_apply(lambda x: get_user_embedding_idf(x, doc_dict))])
user_embeddings.columns = [f'topic_{i}' for i in range(N_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(N_topic)]]
user_embeddings.head(3)

100%|█████████████████████████████████████████████████████████████████████████████| 8000/8000 [00:40<00:00, 197.38it/s]


,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9
0,u105138,0.843277,0.0,0.0,1.448759,1.354633,0.114549,2.956571,0.0,0.000000,0.111062
1,u108690,0.260020,0.0,0.0,3.386782,1.523263,0.391369,1.302075,0.0,0.738087,0.213773
2,u108339,1.227672,0.0,0.0,2.237802,1.745793,1.377893,0.200792,0.0,0.901817,0.115146


In [227]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,churn
0,u105138,0.843277,0.0,0.0,1.448759,1.354633,0.114549,2.956571,0.0,0.000000,0.111062,0
1,u108690,0.260020,0.0,0.0,3.386782,1.523263,0.391369,1.302075,0.0,0.738087,0.213773,1
2,u108339,1.227672,0.0,0.0,2.237802,1.745793,1.377893,0.200792,0.0,0.901817,0.115146,1


In [228]:
# разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(N_topic)]], 
                                                    X['churn'], random_state=0)

In [229]:
model = LogisticRegression()
# обучим 
model.fit(X_train, y_train)

LogisticRegression()

In [230]:
# наши прогнозы для тестовой выборки
preds = model.predict_proba(X_test)[:, 1]
preds[:10]

array([1.10826769e-01, 1.68178133e-02, 8.52098870e-01, 1.91792962e-01,
       2.99623660e-02, 2.88420133e-03, 6.82651627e-02, 2.12295746e-07,
       3.41822244e-01, 9.31779332e-02])

In [231]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')                                                                        

Best Threshold=0.3110400999653355, F-Score=0.729, Precision=0.676, Recall=0.792


In [232]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.9641607070178498

In [233]:
metrics_df = metrics_df.append({
    'model': 'idf_median',
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)

C:\Temp\ipykernel_12412\1015548053.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({



5. Сформировать на выходе единую таблицу, сравнивающую качество 2/3 разных метода получения эмбедингов пользователей: median, max, idf_mean по метрикам roc_auc, precision, recall, f_score


In [234]:
metrics_df

,model,thresh,F-Score,Precision,Recall,ROC AUC
0,mean,0.282171,0.698225,0.675573,0.722449,0.938417
1,median,0.278946,0.685039,0.661597,0.710204,0.940585
2,amax,0.311775,0.588983,0.612335,0.567347,0.906053
3,idf_median,0.311040,0.729323,0.675958,0.791837,0.964161


6. Сделать самостоятельные выводы и предположения о том, почему тот или ной способ оказался эффективнее остальных

После смены разных настроек N_topic, passes
стало понятно что LogisticRegression менее чувствительная модель